In [1]:
from dataloader import ClfDataset,ClfSegDataset, ClfAttentionDataset, get_balanced_loader, get_loader, get_mixup_gen
from mylib.models import densenet, densenetf , resnet,  metrics, losses,densenetf_avr

from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau,LearningRateScheduler
from tensorflow.python.keras.optimizers import Adam
import tensorflow as tf
import os
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
lines = pd.read_csv('train_10fold.csv')

def main(batch_sizes, crop_size, random_move, learning_rate,
         model_path, train_subset, val_subset,
         segmentation_task_ratio, weight_decay, save_folder, epochs):
    '''
    :param batch_sizes: the number of examples of each class in a single batch
    :param crop_size: the input size
    :param random_move: the random move in data augmentation
    :param learning_rate: learning rate of the optimizer
    :param segmentation_task_ratio: the weight of segmentation loss in total loss
    :param weight_decay: l2 weight decay
    :param save_folder: where to save the snapshots, tensorflow logs, etc.
    :param epochs: how many epochs to run
    :return:
    '''
    batch_size = sum(batch_sizes)

    train_dataset = ClfAttentionDataset(crop_size=crop_size, subset=train_subset, move=random_move,lines=lines)

    val_dataset = ClfAttentionDataset(crop_size=crop_size, subset=val_subset, move=None,lines=lines)
    
    g1 = get_balanced_loader(train_dataset, batch_sizes=batch_sizes)
    
    g2 = get_loader(train_dataset, batch_size=batch_size)
    
    train_loader = get_mixup_gen(g1,g2,0.3,batch_size)
    
    val_loader = get_loader(val_dataset, batch_size=batch_size)

    model = densenetf_avr.get_compiled(output_size=2,
                                    optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                                    loss='categorical_crossentropy',
                                    metrics=['accuracy', metrics.precision, metrics.recall, metrics.fmeasure],
                                    weights = model_path,
                                    weight_decay=weight_decay)
    
    def lr_sch(epoch):
        #120 total
        if epoch <10:
            return 1e-3
        if 10<=epoch<30:
            return 3e-4
        if 30<=epoch<50:
            return 1e-4
        if 50<=epoch<70:
            return 1e-5
        if 70<=epoch<90:
            return 1e-6
        if 90<=epoch<110:
            return 1e-7
        if 110<=epoch<120:
            return 5e-8
    
    lr_scheduler = LearningRateScheduler(lr_sch)

    # checkpointer = ModelCheckpoint(filepath='tmp/%s/weights.{epoch:02d}.h5' % save_folder, verbose=1,
    #                                period=1, save_weights_only=True, monitor='val_accuracy')
    
    checkpointer = ModelCheckpoint(filepath='tmp/%s/weights.{epoch:02d}.h5' % save_folder, verbose=1,
                                   period=1, save_weights_only=True, monitor='val_accuracy')
    
    best_keeper = ModelCheckpoint(filepath='tmp/%s/best.h5' % save_folder, verbose=1, save_weights_only=True,
                                  monitor='val_accuracy', save_best_only=True, period=1, mode='auto')
    csv_logger = CSVLogger('tmp/%s/training.csv' % save_folder)
    
    tensorboard = TensorBoard(log_dir='tmp/%s/logs/' % save_folder, profile_batch = 100000000)
    
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, mode='auto',
                                   patience=140, verbose=1)
    lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.334, patience=10,
                                   verbose=1, mode='auto', epsilon=1.e-5, cooldown=2, min_lr=0)

    model.fit_generator(generator=train_loader, steps_per_epoch=len(train_dataset) // batch_size, max_queue_size=500,
                        validation_data=val_loader, epochs=epochs, validation_steps=len(val_dataset) // batch_size,
                        callbacks=[checkpointer, early_stopping, best_keeper, lr_reducer, csv_logger, tensorboard])

Using TensorFlow backend.


if __name__ == '__main__':
    from random import shuffle
    best_model = None
    for j in range(5):
        x = [0,1,2,3,4]       
        shuffle(x)
        for i in x:
            index = list(range(5))
            val_subset = [index.pop(i)]
            train_subset = index
            main(batch_sizes=[2, 2],
                 crop_size=[32, 32, 32],
                 train_subset = train_subset,
                 val_subset = val_subset,
                 model_path = best_model,
                 random_move=3,
                 learning_rate=1.e-4 * (0.333 ** j),
                 segmentation_task_ratio=0.2,
                 weight_decay=0.,
                 save_folder='test',
                 epochs=20)
            best_model = './tmp/test/weights.h5'

for i in range(10):
    index = list(range(10))

    val_subset = [10]
    train_subset = index

    main(batch_sizes=[2, 2],
          crop_size=[32, 32, 32],
          train_subset = train_subset,
          val_subset = val_subset,
          model_path = None,
          random_move=3,
          learning_rate=3e-4,
          segmentation_task_ratio=0.2,
          weight_decay=1e-6,
          save_folder='10foldval(124nols)_%d'%i,
          epochs=110)

In [2]:
main(batch_sizes=[2, 2],
          crop_size=[32, 32, 32],
          train_subset = list(range(9)),
          val_subset = [1],
          model_path = None,
          random_move=4,
          learning_rate=3e-4,
          segmentation_task_ratio=0.2,
          weight_decay=1e-6,
          save_folder='224avrpoolingattentionfGlobalavrk=18dropout',
          epochs=160)

Model hyper-parameters: {'activation': <function <lambda> at 0x000001DE5883BE58>, 'bn_scale': True, 'weight_decay': 1e-06, 'kernel_initializer': 'he_uniform', 'first_scale': <function <lambda> at 0x000001DE58839948>, 'dhw': [32, 32, 32], 'k': 18, 'bottleneck': [2, 2, 4], 'compression': 2, 'first_layer': 32, 'down_structure': [2, 2, 4], 'output_size': 2, 'dropout_rate': None}
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 32,  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 32, 32, 32, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv3d (Conv3

Instructions for updating:
Please use Model.fit, which supports generators.
Size 465
Size 465
  ...
    to  
  ['...']
Size 92
  ...
    to  
  ['...']
Train for 116 steps, validate for 23 steps
Epoch 1/160
115/116 [============================>.] - ETA: 0s - loss: 0.7218 - categorical_crossentropy: 0.7204 - accuracy: 0.5326 - precision: 0.4174 - recall: 0.4174 - fmeasure: 0.4174
Epoch 00001: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.01.h5

Epoch 00001: val_accuracy improved from -inf to 0.46739, saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/best.h5
116/116 [==============================] - 20s 176ms/step - loss: 0.7213 - categorical_crossentropy: 0.7200 - accuracy: 0.5345 - precision: 0.4138 - recall: 0.4138 - fmeasure: 0.4138 - val_loss: 1.0919 - val_categorical_crossentropy: 1.0906 - val_accuracy: 0.4674 - val_precision: 0.4674 - val_recall: 0.4674 - val_fmeasure: 0.4674
Epoch 2/160
115/116 [============================>.] - ETA: 0s -

116/116 [==============================] - 14s 118ms/step - loss: 0.6737 - categorical_crossentropy: 0.6723 - accuracy: 0.5948 - precision: 0.4397 - recall: 0.4397 - fmeasure: 0.4397 - val_loss: 0.6725 - val_categorical_crossentropy: 0.6711 - val_accuracy: 0.6739 - val_precision: 0.6522 - val_recall: 0.6522 - val_fmeasure: 0.6522
Epoch 12/160
115/116 [============================>.] - ETA: 0s - loss: 0.6819 - categorical_crossentropy: 0.6805 - accuracy: 0.5739 - precision: 0.4522 - recall: 0.4522 - fmeasure: 0.4522
Epoch 00012: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.12.h5

Epoch 00012: val_accuracy did not improve from 0.67391
116/116 [==============================] - 13s 115ms/step - loss: 0.6829 - categorical_crossentropy: 0.6815 - accuracy: 0.5690 - precision: 0.4483 - recall: 0.4483 - fmeasure: 0.4483 - val_loss: 0.6610 - val_categorical_crossentropy: 0.6597 - val_accuracy: 0.6522 - val_precision: 0.6413 - val_recall: 0.6413 - val_fmeasure: 0.6413


116/116 [==============================] - 13s 116ms/step - loss: 0.6610 - categorical_crossentropy: 0.6597 - accuracy: 0.6164 - precision: 0.5086 - recall: 0.5086 - fmeasure: 0.5086 - val_loss: 0.6929 - val_categorical_crossentropy: 0.6915 - val_accuracy: 0.6196 - val_precision: 0.5761 - val_recall: 0.5761 - val_fmeasure: 0.5761
Epoch 24/160
115/116 [============================>.] - ETA: 0s - loss: 0.6792 - categorical_crossentropy: 0.6779 - accuracy: 0.5804 - precision: 0.4304 - recall: 0.4304 - fmeasure: 0.4304
Epoch 00024: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.24.h5

Epoch 00024: val_accuracy did not improve from 0.69565
116/116 [==============================] - 14s 122ms/step - loss: 0.6795 - categorical_crossentropy: 0.6781 - accuracy: 0.5797 - precision: 0.4310 - recall: 0.4310 - fmeasure: 0.4310 - val_loss: 0.6673 - val_categorical_crossentropy: 0.6659 - val_accuracy: 0.5652 - val_precision: 0.5326 - val_recall: 0.5326 - val_fmeasure: 0.5326


Epoch 36/160
115/116 [============================>.] - ETA: 0s - loss: 0.6551 - categorical_crossentropy: 0.6537 - accuracy: 0.6391 - precision: 0.4957 - recall: 0.4957 - fmeasure: 0.4957
Epoch 00036: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.36.h5

Epoch 00036: val_accuracy improved from 0.69565 to 0.72826, saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/best.h5
116/116 [==============================] - 14s 121ms/step - loss: 0.6561 - categorical_crossentropy: 0.6547 - accuracy: 0.6358 - precision: 0.4935 - recall: 0.4935 - fmeasure: 0.4935 - val_loss: 0.6209 - val_categorical_crossentropy: 0.6195 - val_accuracy: 0.7283 - val_precision: 0.7174 - val_recall: 0.7174 - val_fmeasure: 0.7174
Epoch 37/160
115/116 [============================>.] - ETA: 0s - loss: 0.6621 - categorical_crossentropy: 0.6607 - accuracy: 0.6217 - precision: 0.4826 - recall: 0.4826 - fmeasure: 0.4826
Epoch 00037: saving model to tmp/224avrpoolingattentionfGlobalavrk

...

Epoch 108/160
115/116 [============================>.] - ETA: 0s - loss: 0.6436 - categorical_crossentropy: 0.6422 - accuracy: 0.6565 - precision: 0.5326 - recall: 0.5326 - fmeasure: 0.5326
Epoch 00108: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.108.h5

Epoch 00108: val_accuracy did not improve from 0.77174
116/116 [==============================] - 13s 112ms/step - loss: 0.6426 - categorical_crossentropy: 0.6412 - accuracy: 0.6595 - precision: 0.5323 - recall: 0.5323 - fmeasure: 0.5323 - val_loss: 0.6296 - val_categorical_crossentropy: 0.6282 - val_accuracy: 0.6848 - val_precision: 0.6630 - val_recall: 0.6630 - val_fmeasure: 0.6630
Epoch 109/160
115/116 [============================>.] - ETA: 0s - loss: 0.6606 - categorical_crossentropy: 0.6592 - accuracy: 0.6152 - precision: 0.5152 - recall: 0.5152 - fmeasure: 0.5152
Epoch 00109: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.109.h5

Epoch 00109: val_accuracy did not improve from 

115/116 [============================>.] - ETA: 0s - loss: 0.6444 - categorical_crossentropy: 0.6430 - accuracy: 0.6543 - precision: 0.5413 - recall: 0.5413 - fmeasure: 0.5413
Epoch 00120: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.120.h5

Epoch 00120: val_accuracy did not improve from 0.77174
116/116 [==============================] - 13s 114ms/step - loss: 0.6452 - categorical_crossentropy: 0.6438 - accuracy: 0.6552 - precision: 0.5431 - recall: 0.5431 - fmeasure: 0.5431 - val_loss: 0.6278 - val_categorical_crossentropy: 0.6264 - val_accuracy: 0.6630 - val_precision: 0.6630 - val_recall: 0.6630 - val_fmeasure: 0.6630
Epoch 121/160
115/116 [============================>.] - ETA: 0s - loss: 0.6636 - categorical_crossentropy: 0.6622 - accuracy: 0.6478 - precision: 0.5130 - recall: 0.5130 - fmeasure: 0.5130
Epoch 00121: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.121.h5

Epoch 00121: val_accuracy did not improve from 0.77174
116/11

115/116 [============================>.] - ETA: 0s - loss: 0.6427 - categorical_crossentropy: 0.6413 - accuracy: 0.6717 - precision: 0.5587 - recall: 0.5587 - fmeasure: 0.5587
Epoch 00132: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.132.h5

Epoch 00132: val_accuracy did not improve from 0.77174
116/116 [==============================] - 14s 117ms/step - loss: 0.6418 - categorical_crossentropy: 0.6404 - accuracy: 0.6724 - precision: 0.5582 - recall: 0.5582 - fmeasure: 0.5582 - val_loss: 0.6435 - val_categorical_crossentropy: 0.6421 - val_accuracy: 0.6413 - val_precision: 0.6196 - val_recall: 0.6196 - val_fmeasure: 0.6196
Epoch 133/160
115/116 [============================>.] - ETA: 0s - loss: 0.6300 - categorical_crossentropy: 0.6287 - accuracy: 0.6978 - precision: 0.5652 - recall: 0.5652 - fmeasure: 0.5652
Epoch 00133: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.133.h5

Epoch 00133: val_accuracy did not improve from 0.77174
116/11

115/116 [============================>.] - ETA: 0s - loss: 0.6587 - categorical_crossentropy: 0.6573 - accuracy: 0.6217 - precision: 0.5087 - recall: 0.5087 - fmeasure: 0.5087
Epoch 00144: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.144.h5

Epoch 00144: val_accuracy did not improve from 0.77174
116/116 [==============================] - 13s 114ms/step - loss: 0.6579 - categorical_crossentropy: 0.6565 - accuracy: 0.6228 - precision: 0.5086 - recall: 0.5086 - fmeasure: 0.5086 - val_loss: 0.6451 - val_categorical_crossentropy: 0.6437 - val_accuracy: 0.6413 - val_precision: 0.6196 - val_recall: 0.6196 - val_fmeasure: 0.6196
Epoch 145/160
115/116 [============================>.] - ETA: 0s - loss: 0.6409 - categorical_crossentropy: 0.6395 - accuracy: 0.6826 - precision: 0.5522 - recall: 0.5522 - fmeasure: 0.5522
Epoch 00145: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.145.h5

Epoch 00145: val_accuracy did not improve from 0.77174
116/11

115/116 [============================>.] - ETA: 0s - loss: 0.6397 - categorical_crossentropy: 0.6383 - accuracy: 0.6739 - precision: 0.5565 - recall: 0.5565 - fmeasure: 0.5565
Epoch 00156: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.156.h5

Epoch 00156: val_accuracy did not improve from 0.77174
116/116 [==============================] - 14s 117ms/step - loss: 0.6399 - categorical_crossentropy: 0.6385 - accuracy: 0.6746 - precision: 0.5560 - recall: 0.5560 - fmeasure: 0.5560 - val_loss: 0.6680 - val_categorical_crossentropy: 0.6666 - val_accuracy: 0.6304 - val_precision: 0.6196 - val_recall: 0.6196 - val_fmeasure: 0.6196
Epoch 157/160
115/116 [============================>.] - ETA: 0s - loss: 0.6517 - categorical_crossentropy: 0.6503 - accuracy: 0.6478 - precision: 0.5565 - recall: 0.5565 - fmeasure: 0.5565
Epoch 00157: saving model to tmp/224avrpoolingattentionfGlobalavrk=18dropout/weights.157.h5

Epoch 00157: val_accuracy did not improve from 0.77174
116/11